# Multiplier TEST code

fixed number to stochastic conversion & multiply & stochastic to fixed number conversion

## Import

In [1]:
import random, struct, math
import torch
import numpy as np
import torch.nn as nn 
import argparse
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## Parser

In [2]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=16, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=7, help='Number of bit width')
args = parser.parse_args(args=[])

## Definition

### int2bin : int number to binary number

In [3]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

### XOR : xor gate with string

In [4]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

### SNUM : sign number determination

In [5]:
def snum(a):
    if a >= 0 :
        return '0'
    else :
        return '1'

## binInv : binary Inversion

In [6]:
def binInv(bIn):
    bOut = bin(int(bIn,2)^(pow(2,len(bIn))-1)).replace('0b','').rjust(len(bIn),'0')
    return bOut

### fxp & flp2fix : floating number to fixed number conversion

In [7]:
class fxp:
    def __init__(self, bIn, iBWF):
        self.iFullBW = len(bIn)
        self.iIntgBW = self.iFullBW - iBWF
        self.bSign = bIn[0]
        self.bIntg = bIn[:self.iIntgBW]
        self.bFrac = bIn[self.iIntgBW:]
        self.fFull = 0
        try:
            for idx, bit in enumerate(bIn):
                if idx == 0:
                    self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
                else:
                    self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
        except:
            print(bIn)
        self.dispFull = self.bIntg +"."+ self.bFrac 
        return

In [8]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol = 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

        self.iFLP2INT = abs(int(fIn * 2 ** iBWF))
        if fIn < 0:
            self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

        if fIn >= 0:
            self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
        else:
            self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
            if len(self.bFull) > iBW:
                self.bFull = '0' * iBW

        self.cssFxp = fxp(self.bFull, self.iBWF)
        self.bSign = self.cssFxp.bSign
        self.bIntg = self.cssFxp.bIntg
        self.bFrac = self.cssFxp.bFrac
        self.fFull = self.cssFxp.fFull
        return

In [9]:
def flp2fixTensor(fIn, iBW, iBWF):
    fMin = - 2 ** (iBW - iBWF - 1)
    fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
    fList = []
    for aTensor in fIn.view(-1):
        fList.append(flp2fix(aTensor, iBW, iBWF).fFull)
    return torch.tensor(fList).view(fIn.size())

### LFSR : make pseudorandom number bitstream

In [10]:
class LFSR7:
    def Random(self):
        self.b0 = eval(f'str(random.randint(0,1))')
        self.b1 = eval(f'str(random.randint(0,1))')
        self.b2 = eval(f'str(random.randint(0,1))')
        self.b3 = eval(f'str(random.randint(0,1))')
        self.b4 = eval(f'str(random.randint(0,1))')
        self.b5 = eval(f'str(random.randint(0,1))')
        self.b6 = eval(f'str(random.randint(0,1))')
        if int(self.b0) + int(self.b1) + int(self.b2) + int(self.b3) + int(self.b4) + int(self.b5) + int(self.b6) == 0 :
            self.b0 = eval(f'str(random.randint(0,1))')
            self.b1 = eval(f'str(random.randint(0,1))')
            self.b2 = eval(f'str(random.randint(0,1))')
            self.b3 = eval(f'str(random.randint(0,1))')
            self.b4 = eval(f'str(random.randint(0,1))')
            self.b5 = eval(f'str(random.randint(0,1))')
            self.b6 = eval(f'str(random.randint(0,1))')
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Normal(self,stream):
        self.b0 = XOR(int(stream[5]),int(stream[6]))
        self.b1 = stream[0]
        self.b2 = stream[1]
        self.b3 = stream[2]
        self.b4 = stream[3]
        self.b5 = stream[4]
        self.b6 = stream[5]
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6
    
    def Allzero(self):
        self.b0 = '0'
        self.b1 = '0'
        self.b2 = '0'
        self.b3 = '0'
        self.b4 = '0'
        self.b5 = '0'
        self.b6 = '0'
        
        return self.b0 + self.b1 + self.b2 + self.b3 + self.b4 + self.b5 + self.b6

### LFSRlist : make pseudorandom number bitstream with 2**bBW cycle

In [11]:
def LFSRlist7():
    lfsr = LFSR7()
    lfsrlist = []
    for k in range(2**(args.bBW)-1): #lfsr number generating
        if k == 0:
            lfsrlist.append(lfsr.Random())
        else :
            lfsrlist.append(lfsr.Normal(lfsrlist[k-1]))
        if (k == 2**(args.bBW)-2):
            lfsrlist.append(lfsr.Allzero())
    
    if (args.bBW) != args.frac_bits :
        if args.bBW < args.frac_bits :
            for i in range(len(lfsrlist)):
                lfsrlist[i] = lfsrlist[i] + (args.frac_bits-args.bBW)*'0'
        else :
            print("it can't work")
            return 0
    
    return lfsrlist

### Comp : Comparator in SNG

In [12]:
def Comp(a,lfsr,snum):
    for com in range(0,len(a)):
        oA = '0'
        if a[com]!=lfsr[com]:
            if(int(a[com]) > int(lfsr[com])):
                oA = '1'
            break
    return XOR(oA,snum)

### Perm : module for permutation SNG 

In [13]:
def perm(a):
    al = len(a)
    blist = []
    if args.frac_bits == args.bBW : 
        for i in range(al) :
            blist.append(a[al-i-1])
    elif args.frac_bits > args.bBW :
        for i in range(al-(args.frac_bits-args.bBW)) :
            blist.append((a[al-(args.frac_bits-args.bBW)-i-1]))
    b = "".join(blist)
    b = b + ('0'*(args.frac_bits-args.bBW))
    return b

### findMaxMin : find abs max value in act tensor

In [14]:
def findMaxMin(data):
    max = torch.max(data)
    min = torch.min(data)
    SF=torch.max(abs(max),abs(min)).item()
    return SF

### SNG : stochastic number generator module

In [15]:
def SNG(iIN,lfsr):

    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating
        lNUM = lfsr[k]
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

### SNG_P : permutation stochastic number generator module

In [16]:
def SNG_P(iIN,lfsr):
    sNUM = snum(iIN)
    
    bIN = flp2fix(iIN,args.full_bits,args.frac_bits).bFull
    bFRAC = bIN[-(args.frac_bits):]
    if sNUM == 1 :
        bFRAC = bin(int(binInv(bFRAC),2)+1).replace('0b','').rjust(args.bBW,'0')
    oAlist = []
    
    for k in range(2**(args.bBW)): #lfsr number generating    
        lNUM = perm(lfsr[k])
        a = Comp(bFRAC,lNUM,sNUM)
        oAlist.append(a) #comparator of input a
    
    oAlist.insert(0,sNUM)
    sA = "".join(oAlist)
    
    if bIN == args.full_bits*'0' :
        return '0'*(2**(args.bBW)+1)
    else :
        return sA

### SNGnumpy : SNG module with numpy

In [17]:
def SNGnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG(float(aNumpy),lfsr))
                     
    return np.array(sList).reshape(fIn.size())

### SNGpnumpy : permutation SNG module with numpy

In [18]:
def SNGpnumpy(fIn,lfsr):
    sList = []
    for aNumpy in fIn.view(-1):
        sList.append(SNG_P(float(aNumpy),lfsr))
                     
    return  np.array(sList).reshape(fIn.size())

### CountOne : count 1 in stochastic number bit stream

In [19]:
def CountOne(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        n = 0
        for a in num:
            if a == '1' :
                n += 1
        if num[0] == '1' :
            nlist.append(n-1)
        else :
            nlist.append(n)
    return torch.tensor(nlist).view(nIn.shape)

### defSign : sign determination in S2N 

In [20]:
def defSign(nIn):
    nlist = []
    for num in nIn.reshape(-1):
        if num[0] == '1' :
            nlist.append(-1)
        else :
            nlist.append(1)
    return torch.tensor(nlist).view(nIn.shape)

### Multiplier

In [21]:
def mul(a,b):
    al = len(a)
    bl = len(b)
    
    outlist = []
    
    if al != bl :
        print("length of string is different")
        return 0
    
    outlist.append(XOR(a[0],b[0]))
    
    for i in range(al-1) :
        outlist.append(str(int(a[i+1]) & int(b[i+1])))
    
    #print(outlist)
    out = "".join(outlist)
    
    return out    

In [22]:
def defSign1(nIn):
    if nIn[0] == '1' :
        return -1
    else :
        return 1

In [23]:
def CountOne1(nIn):
    n = 0
    for num in nIn:
        if num == '1' :
            n += 1
    if nIn[0] == '1' :
        return n-1
    else :
        return n

In [24]:
def S2None(sIn,SF):
    s = defSign1(sIn)
    o = (CountOne1(sIn)/(2**args.bBW))*SF*s
    return o

In [25]:
def mulNumpy(aIn,bIn,aSF,wSF):
    mList = []
    for i in range(aIn.shape[0]):
        for j in range(bIn.T.shape[1]):
            sum = 0
            for k in range(aIn.shape[1]):
                sum += S2None(mul((aIn[i][k].astype(str)),(bIn.T)[k][j].astype(str)),aSF*wSF)
            mList.append(sum)
    return torch.tensor(mList).view(aIn.shape[0],bIn.T.shape[1])

In [26]:
def FindError(a,b):
    elist = []
    for anum,bnum in zip(a.view(-1),b.view(-1)):
        e = (abs(bnum-anum)*100).item()
        elist.append(e)
    return torch.tensor(elist).reshape(a.size())

# FC layer

In [27]:
def fc(aIn,bIn):
    out = torch.matmul(aIn,bIn.t())
    return out

# TEST : LFSR 7 bit

In [28]:
act = torch.rand((10,15))-torch.randint(5,(10,15))+torch.randint(5,(10,15))
wei = torch.rand((10,15))-torch.randint(5,(10,15))+torch.randint(5,(10,15))

In [29]:
print(act)

tensor([[-0.8614, -0.8044, -0.7357,  2.0421, -2.6428, -0.6090, -0.1689,  1.5182,
         -1.7298, -1.7538,  1.4324,  1.6648, -0.7678,  0.7071, -2.2304],
        [ 0.1408,  2.7688, -2.7196,  0.4878, -2.6609,  2.2840, -0.4772,  0.4100,
         -1.2546,  0.2822, -0.9885,  0.5556,  2.2889,  0.0431, -3.4947],
        [ 1.5013, -1.1554, -2.3211, -2.8488, -3.2711,  0.8157,  2.8162, -1.6564,
          3.0228,  2.3369, -1.5484,  1.1102, -1.4387,  3.9506, -1.9146],
        [-0.5203, -1.8882, -3.8050,  3.9029,  0.6186, -1.6763, -2.1122, -0.5420,
         -0.6595,  0.7192, -2.9799, -0.1874, -2.3820, -0.3934, -2.2853],
        [ 0.0513,  2.6443,  0.8183, -1.4914,  1.7596,  2.5000, -2.4687,  4.8590,
          4.6708,  3.4636,  1.8609, -1.4690, -3.3426, -0.7574, -2.3322],
        [ 3.5905,  0.1536, -0.1188,  4.4931, -0.5745,  0.5719, -3.9102, -1.4951,
          0.8339,  2.1615, -0.6006,  2.7981,  1.6680, -2.2760, -0.0270],
        [-0.9548,  1.4735, -3.4389, -0.5400,  0.1944,  3.9823, -0.3866,  3.6

In [30]:
print(wei)

tensor([[-2.5053e+00, -1.0698e+00,  1.1286e+00, -1.5828e+00,  4.1102e-01,
         -3.5315e+00, -1.6925e+00,  2.8489e-01, -2.9325e+00,  3.3535e+00,
          1.7221e+00,  4.0092e+00, -2.1529e+00,  1.1261e+00, -1.1460e-01],
        [ 3.1122e+00,  2.5008e+00,  3.4723e-01,  5.1386e-01,  4.9498e-01,
          9.5524e-02, -7.9540e-01,  9.0379e-01,  2.5149e+00, -8.7080e-01,
          3.8315e-01,  2.7343e+00, -3.5191e-01,  1.3620e+00,  5.6243e-01],
        [-3.9652e+00,  2.7907e-01,  3.3687e+00, -2.6016e+00, -3.6676e-01,
         -8.1022e-01, -1.8222e+00, -7.5851e-02,  1.8441e+00,  1.1708e+00,
         -1.0394e-01,  2.4734e+00, -3.3375e+00,  7.5191e-01, -1.5615e-01],
        [ 2.1602e+00,  2.3404e+00, -3.4765e-01, -5.5353e-01, -2.6145e+00,
         -3.1290e+00,  3.9532e+00,  8.4729e-01,  1.6484e+00, -1.9629e+00,
          1.3868e-01,  2.5222e+00,  9.5968e-02,  1.3623e+00,  1.7605e+00],
        [ 4.7020e+00,  2.6084e+00,  5.5257e-01,  2.5245e-01,  1.1148e+00,
          2.4843e+00, -3.5008e-01,

In [31]:
o = fc(act,wei)

In [32]:
print(o)
print(o.size())

tensor([[ 11.7761,  -1.5014,  -0.7767,   6.7680, -16.6314,  21.2140,   9.8833,
         -16.5407,   7.0815,  12.1741],
        [-14.7194,   1.3468, -17.8198,  -1.5010,   8.9342,   3.7297,   5.6933,
          -4.0163,  13.1148,  -1.9052],
        [ -1.5768,   7.0545,   8.1383,  23.5065,  13.8015,  16.4104, -19.4219,
          16.8575,  12.9191,   8.9693],
        [  5.8566,  -9.2425,  -8.9310, -19.7345,  -7.6151,  16.2440, -13.6257,
           1.0644,   5.3319, -15.6428],
        [ -0.3728,  18.0148,  28.4222, -19.2295,  16.6006,   4.3065, -17.4772,
          10.6156,  11.0153,  28.7852],
        [ -3.6331,  19.2888, -15.5206, -10.2241,  21.5629,  38.4809,  25.3306,
          15.0971, -20.7066,   0.4541],
        [-16.6603,   8.1153,  -7.0491,  -3.6901,   1.2840,  -8.3322,  -2.8042,
          -3.3511,  22.7474,  25.4658],
        [ -4.7225,   8.6248, -25.4318,  28.2258,  13.5002,   7.3632,  17.9418,
          -9.0672,  20.1587, -32.7079],
        [ -6.3921,  13.1546,  -9.8410,  13.2282,

In [33]:
act_f = flp2fixTensor(act,args.full_bits,args.frac_bits)
wei_f = flp2fixTensor(wei,args.full_bits,args.frac_bits)

### number to stochastic conversion

In [34]:
aSF = findMaxMin(act_f)
wSF = findMaxMin(wei_f)

In [35]:
def N2S(act,wei,aSF,wSF):
    
    lfsrlist = LFSRlist7()
    
    a = SNGnumpy(act/aSF,lfsrlist)
    w = SNGpnumpy(wei/wSF,lfsrlist)
    
    return a,w

In [36]:
def N2Sone(data,SF):
    lfsrlist = LFSRlist7()
    
    o = SNGnumpy(data/SF,lfsrlist)
    
    return o

### stochastic to number conversion

In [37]:
a,w = N2S(act,wei,aSF,wSF)

In [38]:
out = mulNumpy(a,w,aSF,wSF)

In [39]:
print(out.size())

torch.Size([10, 10])


In [40]:
#error = FindError(cc,act_f*wei_f)

In [41]:
#print(f'error mean value is : {torch.mean(error)}')
#print(f'------------------------------------------------------------------------------------------')
#for i in range(100):
#    for j in range(150):
#        print(f'act value is            : {act_f[i][j]:^25}( --> {ca[i][j]})')
#        print(f'wei value is            : {wei_f[i][j]:^25}( --> {cw[i][j]})')
#        print(f'stochastic mul value is : {cc[i][j]:^25}( --> {ca[i][j] * cw[i][j]})')
#        print(f'fixed mul value is      : {act_f[i][j]*wei_f[i][j]:^25}')
#        print(f'error is                : {error[i][j]:^25}')
#        print(f'------------------------------------------------------------------------------------------')

In [42]:
print(f'------------------------------------------------------------------------------------------')
for i in range(10):
    for j in range(10):
        print(f'stochastic FC value is            : {out[i][j]:^25}')
        print(f'original FC value is              : {o[i][j]:^25}')
        print(f'------------------------------------------------------------------------------------------')

------------------------------------------------------------------------------------------
stochastic FC value is            :    13.913833618164062    
original FC value is              :    11.776119232177734    
------------------------------------------------------------------------------------------
stochastic FC value is            :     0.927588939666748    
original FC value is              :     -1.50139319896698    
------------------------------------------------------------------------------------------
stochastic FC value is            :    1.2986245155334473    
original FC value is              :    -0.7767124176025391   
------------------------------------------------------------------------------------------
stochastic FC value is            :     9.64692497253418     
original FC value is              :     6.767990589141846    
------------------------------------------------------------------------------------------
stochastic FC value is            :    -13.542798

## ERROR

In [43]:
def MSE(a,b):
    s = 0
    n = 0

    for i in range(a.size()[0]):
        for j in range(b.size()[0]):
            square = (((a[i][j])-b[i][j])**2).item()
            s += square
            n += 1
    error = s/n
    return error

## ERROR : 7bit LFSR

In [44]:
MSE(out,o)

17.035885625332593